In [28]:
%reset -f

from pprint import pprint
from IPython.display import Image

import replicate
import requests

def test_api_call():
    return replicate.run(
      model_version="stability-ai/stable-diffusion:27b93a2413e7f36cd83da926f3656280b2931564ff050bf9575f1fdf9bcd7478",
      input={"prompt": "a 19th century portrait of a wombat gentleman"}
    )

def run_replicate(**kwargs):
    return replicate.run(**kwargs)

def download_img(url):
    filename = url.split('/')[-1]
    response = requests.get(url)
    pprint(dict(response.headers))

    if response.headers['Content-Type'] == 'image/png':
        with open(filename, 'wb') as f:
            f.write(response.content)
            print('Image downloaded successfully.')
            Image(filename=filename)
    else:
        print('The URL does not contain a PNG image.')
    pass

def show_img_from_url(url):
    display(Image(url=url))

def main():
    # prompt_str = "a 19th century portrait of a wombat gentleman"

    prompt_str = """neuromancer scene of of future cyberpunk 
    book character, jacked into computer, realistic, virtual, cyberspace, 
    portrait, hacker"""

    replicate_inputs = {
        "model_version":"stability-ai/stable-diffusion:27b93a2413e7f36cd83da926f3656280b2931564ff050bf9575f1fdf9bcd7478",
        "input": {"prompt": prompt_str},
    }

    url_list = run_replicate(**replicate_inputs)
    print(url_list)
    url = url_list[0]
    display(Image(url=url))

['https://replicate.delivery/pbxt/df7lbHWjpx16UCoF6PiA7eoe1wVLrfQUAgglDh8Yd0781mKDB/out-0.png']


In [26]:
%reset -f

import os
import random
from pprint import pprint
import uuid

import replicate
import requests
from IPython.display import Image
from dotenv import load_dotenv

load_dotenv()

# for key, value in os.environ.items():
#     print(f'{key}={value}')

# unique_id = uuid.uuid4()
# print(unique_id)

class replicateApi:
    def __init__(self):
        self.username = 'dummy-work-account'
        self.model_name = 'yourmodel'
        self.model_slug = f'{self.username}/{self.model_name}'
        self.img_urls = []
        self.results = {}
        self.model = replicate.models.get(self.model_slug)
        self.versions = self.model.versions.list()

    def generate_dreambooth_img(self, prompt):
        if not len(self.versions) == 1:
            raise ValueError('unexpected number of model versions')
        else:
            version = self.versions[0]
        negative_prompt = "blurry, deformed, watermark, dark lighting, image caption, low quality, low resolution, malformed, messy, blurry, watermark"
        image_urls = version.predict(
            prompt=prompt,
            width=512,
            height=512,
            negative_prompt=negative_prompt,
            num_outputs=4,
        )
        self.img_urls.append(image_urls)
        self.results[str(uuid.uuid4())] = {"prompt":prompt,"img_url_output":image_urls}
    
    def img_urls_from_results(self):
        self.img_urls = [resultdict.get('img_url_output', None) for resultid, resultdict in self.results.items()]
    
    def get_img_urls(self):
        if len(self.img_urls) == 0 and len(self.results) == 0:
            raise ValueError('no results present')
        elif len(self.img_urls) == 0:
            self.img_urls_from_results()
        return [item for sublist in self.img_urls for item in sublist]
    
    def execute_prompt_list(self, prompt_list):
        random.shuffle(prompts)
        for prompt in prompts:
            print(prompt)
            self.generate_dreambooth_img(prompt)
            print('complete')
        print('prompt list complete')
        # pprint(self.results)
    
    def display_imgs(self, url):
        display(Image(url=url))
    
    def display_all_imgs(self):
        for url in self.get_img_urls():
            self.display_imgs(url)
            
    def download_img(self, url):
        filename = url.split('/')[-1]
        response = requests.get(url)
        pprint(dict(response.headers))

        if response.headers['Content-Type'] == 'image/png':
            with open(filename, 'wb') as f:
                f.write(response.content)
                print('Image downloaded successfully.')
                Image(filename=filename)
        else:
            print('The URL does not contain a PNG image.')
        pass

In [27]:
subject_id = "cjw"
subject_noun = "happy male human"
subject_gender = "his"
# these prompts were all taken from [https://lexica.art/?q=dog+portrait](https://lexica.art/?q=dog+portrait) 
prompts = [
    f"Adorably cute {subject_id} {subject_noun} portrait, artstation winner by Victo Ngai, Kilian Eng and by Jake Parker, vibrant colors, winning-award masterpiece, fantastically gaudy, aesthetic octane render, 8K HD Resolution",
    f"Incredibly cute {subject_id} {subject_noun} portrait, artstation winner by Victo Ngai, Kilian Eng and by Jake Parker, vibrant colors, winning-award masterpiece, fantastically gaudy, aesthetic octane render, 8K HD Resolution",
    f"a high quality painting of a very cute {subject_id} {subject_noun}, friendly, curious expression. painting by artgerm and greg rutkowski and alphonse mucha",
    f"magnificent {subject_id} {subject_noun} portrait masterpiece work of art. oil on canvas. Digitally painted. Realistic. 3D. 8k. UHD.",
    f"intricate five star {subject_id} {subject_noun} facial portrait by casey weldon, oil on canvas, hdr, high detail, photo realistic, hyperrealism, matte finish, high contrast, 3 d depth, centered, masterpiece, vivid and vibrant colors, enhanced light effect, enhanced eye detail, artstationhd ",
    f"a portrait of a {subject_id} {subject_noun} in a scenic environment by mary beale and rembrandt, royal, noble, baroque art, trending on artstation ",
    f"a painted portrait of a {subject_id} {subject_noun} with brown fur, no white fur, wearing a sea captain's uniform and hat, sea in background, oil painting by thomas gainsborough, elegant, highly detailed, anthro, anthropomorphic {subject_noun}, epic fantasy art, trending on artstation, photorealistic, photoshop, behance winner ",
    f"{subject_id} {subject_noun} guarding {subject_gender} home, dramatic sunset lighting, mat painting, highly detailed, ",
    f"{subject_id} {subject_noun}, realistic shaded lighting poster by ilya kuvshinov katsuhiro otomo, magali villeneuve, artgerm, jeremy lipkin and michael garmash and rob rey ",
    f"a painting of a {subject_id} {subject_noun} {subject_noun}, greg rutkowski, cinematic lighting, hyper realistic painting",
    f"painting of {subject_id} by andy warhol",
    f"abstract impressionist portrate of {subject_id}",
    f"cartoon portrait of {subject_id} facial, high definition, happy",
]

api = replicateApi()
api.execute_prompt_list(prompt_list=prompts)

a portrait of a cjw happy male human in a scenic environment by mary beale and rembrandt, royal, noble, baroque art, trending on artstation 
complete
a painting of a cjw happy male human happy male human, greg rutkowski, cinematic lighting, hyper realistic painting
complete
cjw happy male human guarding his home, dramatic sunset lighting, mat painting, highly detailed, 
complete
intricate five star cjw happy male human facial portrait by casey weldon, oil on canvas, hdr, high detail, photo realistic, hyperrealism, matte finish, high contrast, 3 d depth, centered, masterpiece, vivid and vibrant colors, enhanced light effect, enhanced eye detail, artstationhd 
complete
cjw happy male human, realistic shaded lighting poster by ilya kuvshinov katsuhiro otomo, magali villeneuve, artgerm, jeremy lipkin and michael garmash and rob rey 
complete
magnificent cjw happy male human portrait masterpiece work of art. oil on canvas. Digitally painted. Realistic. 3D. 8k. UHD.
complete
a painted portra